In [30]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

import re
import nltk
import string
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    return res

positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))    
        
read_lexicon()
opinion_lexicon = positive_lexicon + negative_lexicon

In [34]:
import pandas as pd
df = pd.read_csv('Results/Opinion/PRICES.csv')
# df.head()

In [35]:
def run(name):
    df = pd.read_csv('Results/Opinion/'+ name +'.csv')
    sf = pd.DataFrame(columns = ['id','opinion', 'sentiment', 'type'])

    for index in range(0, len(df)):
        opini = df['opini'][index]
        sentimen = df['sentimen'][index]
        tipe = '-'

        if opini == 'high':
            print(opini, sentimen)
        Flag = False
        if sentimen == 'positive':
            if opini not in positive_lexicon and opini in negative_lexicon:
#                 print(opini, '+')
                Flag = True
            elif opini not in opinion_lexicon:
#                 print(opini, '+ baru')
                Flag = True
                tipe = 'baru'

        elif sentimen == 'negative':
            if opini not in negative_lexicon and opini in positive_lexicon:
#                 print(opini, '-')
                Flag = True
            elif opini not in opinion_lexicon:
#                 print(opini, '- baru')
                Flag = True
                tipe = 'baru'

        if Flag:
            sf = sf.append({'id': index, 
                    'opinion': opini,
                    'sentiment': sentimen,
                    'type': tipe,
                   }, ignore_index=True)

    sf.to_csv('Results/Opinion/beda-' + name + '.csv')
    sf.to_excel('Results/Opinion/beda-' + name + '.xlsx')

In [36]:
run('AMBIENCE')
run('FOOD')
run('SERVICE')
run('PRICES')

high positive
high negative


In [29]:
'high' in opinion_lexicon

False